In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import backend as k
from keras import losses
from keras import optimizers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Flatten
from keras.models import Sequential
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data

--2022-10-06 20:31:29--  https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 702942 (686K) [application/x-httpd-php]
Saving to: ‘spambase.data.1’

spambase.data.1     100%[===================>] 686.47K   891KB/s    in 0.8s    

2022-10-06 20:31:30 (891 KB/s) - ‘spambase.data.1’ saved [702942/702942]



In [3]:
def load_spam_dataset(spamBase_data):
  spamBase_data = pd.read_csv(spamBase_data, header=None)
  label = spamBase_data[57]
  spamBase_data.drop(spamBase_data.columns[[57]], axis=1, inplace=True)
  features = spamBase_data
  x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.2)

  spamBase_data_mean = x_train.mean(axis=0)
  spamBase_data_std = x_train.std(axis=0)

  x_train = x_train - spamBase_data_mean
  x_train = x_train / spamBase_data_std
  x_test = x_test - spamBase_data_mean
  x_test = x_test / spamBase_data_std

  print(x_train.shape)
  print(x_test.shape)
  data = x_train, y_train, x_test, y_test
  return data


data_loader = load_spam_dataset('/content/spambase.data')


(3680, 57)
(921, 57)


In [4]:
inputs = keras.Input(shape=(data_loader[0].shape[1]), name="digits")
x1 = layers.Dense(32, activation="tanh")(inputs)
x1 = layers.Dense(16, activation="tanh")(x1)
x1 = layers.Dense(8, activation="tanh")(x1)
x1 = layers.Dense(4, activation="tanh")(x1)
x1 = layers.Dense(2, activation="tanh")(x1)

outputs = layers.Dense(1, name="predictions")(x1)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 digits (InputLayer)         [(None, 57)]              0         
                                                                 
 dense (Dense)               (None, 32)                1856      
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
 dense_3 (Dense)             (None, 4)                 36        
                                                                 
 dense_4 (Dense)             (None, 2)                 10        
                                                                 
 predictions (Dense)         (None, 1)                 3     

In [5]:
x_train = data_loader[0]
x_test = data_loader[2]
y_train = data_loader[1]
y_test = data_loader[3]
x_train = np.reshape(x_train, (-1, 57))
x_test = np.reshape(x_test, (-1, 57))

x_val = x_train[-3680:]
y_val = y_train[-3680:]
x_train = x_train[:-921]
y_train = y_train[:-921]

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
train_acc_metric = keras.metrics.BinaryAccuracy()
val_acc_metric = keras.metrics.BinaryAccuracy()
batch_size = 64


train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)
# val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = tf.data.Dataset.from_tensors((x_val, y_val))



In [6]:
epochs = 40
for epoch in range(epochs):
    print("\nEpoch-%d" % (epoch,))
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True) # Run the forward pass of the layer.
            loss_value = loss_fn(y_batch_train, logits)

        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        if step % 5 == 0:
            print(
                "Training loss at step-%d: %.4f"
                % (step, float(loss_value))
            )
    model.save("model" + str(epoch + 1) + ".h5")


Epoch-0
Training loss at step-0: 0.7119
Training loss at step-5: 0.6655
Training loss at step-10: 0.6246
Training loss at step-15: 0.5900
Training loss at step-20: 0.6012
Training loss at step-25: 0.5673
Training loss at step-30: 0.5722
Training loss at step-35: 0.5128


Training loss at step-40: 0.5088

Epoch-1
Training loss at step-0: 0.5084
Training loss at step-5: 0.4681
Training loss at step-10: 0.4634
Training loss at step-15: 0.3940
Training loss at step-20: 0.4481
Training loss at step-25: 0.4112
Training loss at step-30: 0.4004
Training loss at step-35: 0.3775
Training loss at step-40: 0.3499



Epoch-2
Training loss at step-0: 0.3596
Training loss at step-5: 0.3789
Training loss at step-10: 0.3287
Training loss at step-15: 0.3707
Training loss at step-20: 0.4275
Training loss at step-25: 0.3115
Training loss at step-30: 0.3560
Training loss at step-35: 0.3282
Training loss at step-40: 0.3097



Epoch-3
Training loss at step-0: 0.3642
Training loss at step-5: 0.3537
Training loss at step-10: 0.3598
Training loss at step-15: 0.2850
Training loss at step-20: 0.4181
Training loss at step-25: 0.2617
Training loss at step-30: 0.2742
Training loss at step-35: 0.3404
Training loss at step-40: 0.3541



Epoch-4
Training loss at step-0: 0.2954
Training loss at step-5: 0.3450
Training loss at step-10: 0.2608
Training loss at step-15: 0.2820
Training loss at step-20: 0.2406
Training loss at step-25: 0.3634
Training loss at step-30: 0.3331
Training loss at step-35: 0.2668
Training loss at step-40: 0.2814


...


Epoch-36
Training loss at step-0: 0.1289
Training loss at step-5: 0.0968
Training loss at step-10: 0.0885
Training loss at step-15: 0.1293
Training loss at step-20: 0.1511


Training loss at step-25: 0.1302
Training loss at step-30: 0.1604
Training loss at step-35: 0.1402
Training loss at step-40: 0.1103

Epoch-37
Training loss at step-0: 0.1089
Training loss at step-5: 0.1477
Training loss at step-10: 0.1652
Training loss at step-15: 0.0674
Training loss at step-20: 0.1215
Training loss at step-25: 0.1015
Training loss at step-30: 0.0614
Training loss at step-35: 0.1228
Training loss at step-40: 0.0478



Epoch-38
Training loss at step-0: 0.1356
Training loss at step-5: 0.1488
Training loss at step-10: 0.1043
Training loss at step-15: 0.1333
Training loss at step-20: 0.0735


Training loss at step-25: 0.1477
Training loss at step-30: 0.1541
Training loss at step-35: 0.2056
Training loss at step-40: 0.2352

Epoch-39
Training loss at step-0: 0.1289
Training loss at step-5: 0.1594
Training loss at step-10: 0.1391
Training loss at step-15: 0.0829
Training loss at step-20: 0.1512
Training loss at step-25: 0.1869


Training loss at step-30: 0.1691
Training loss at step-35: 0.0611
Training loss at step-40: 0.1483


In [7]:
from keras.models import load_model
model = load_model('./model39.h5')
pred = model.predict(x_test)
print("Prediction vector: ",pred)

Prediction vector:  [[ 2.9739454 ]
 [ 2.367807  ]
 [-3.3956094 ]
 [-2.3202722 ]
 [-3.3956335 ]
 [-3.3959103 ]
 [ 2.9785826 ]
 [ 2.9738898 ]
 [ 2.9715672 ]
 [ 2.9795914 ]
 [ 2.9766393 ]
 [ 2.9789407 ]
 [-3.0243611 ]
 [-3.3966296 ]
 [-3.3450406 ]
 [ 2.4461844 ]
 [-3.395797  ]
 [-3.3964083 ]
 [-3.3960652 ]
 [ 2.9795368 ]
 [-3.3967748 ]
 [-3.3648465 ]
 [ 2.9524043 ]
 [ 2.976942  ]
 [-3.3967512 ]
 [-3.3968084 ]
 [ 2.9805486 ]
 [-3.3967392 ]
 [-3.3905194 ]
 [-3.396192  ]
 [ 2.9772365 ]
 [-3.3936486 ]
 [ 2.8420234 ]
 [-2.3240395 ]
 [-3.3936365 ]
 [ 2.9766765 ]
 [-3.3965003 ]
 [ 2.7980578 ]
 [-3.3967283 ]
 [-3.396619  ]
 [-3.394197  ]
 [ 2.978348  ]
 [ 2.9814463 ]
 [-3.395541  ]
 [-3.396676  ]
 [ 2.964066  ]
 [-3.3906739 ]
 [-3.3960059 ]
 [ 2.9776504 ]
 [-2.387347  ]
 [ 2.981261  ]
 [-1.4682854 ]
 [-3.2801    ]
 [ 2.9772863 ]
 [-3.3899724 ]
 [-0.87323827]
 [ 2.9760973 ]
 [ 2.9684262 ]
 [ 2.971897  ]
 [ 2.9489398 ]
 [-3.391882  ]
 [ 2.9469395 ]
 [ 2.9805903 ]
 [-3.3962057 ]
 [-3.3966274 ]
 [ 2.

In [8]:
epochs = 40
for epoch in range(epochs):
    print("\nEpoch-%d" % (epoch,))

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        train_acc_metric.update_state(y_batch_train, logits)
        if step % 10 == 0:
            print(
                "Training loss at step-%d: %.4f"
                % (step, float(loss_value))
            )

    train_acc = train_acc_metric.result()
    print("Training acc over Epoch-%d: %.4f" % (epoch,float(train_acc),))
    train_acc_metric.reset_states()
    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        # Update val metrics
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    
    print("Test accuracy: %.4f" % (float(val_acc),))
    # model.save("model_new" + str(epoch + 1) + ".h5")


Epoch-0
Training loss at step-0: 0.0613
Training loss at step-10: 0.0775
Training loss at step-20: 0.0963
Training loss at step-30: 0.2506
Training loss at step-40: 0.1147
Training acc over Epoch-0: 0.9592
Test accuracy: 0.9465

Epoch-1
Training loss at step-0: 0.2108
Training loss at step-10: 0.1515
Training loss at step-20: 0.0719
Training loss at step-30: 0.1536
Training loss at step-40: 0.0700
Training acc over Epoch-1: 0.9599
Test accuracy: 0.9625

Epoch-2
Training loss at step-0: 0.1108
Training loss at step-10: 0.0769
Training loss at step-20: 0.1694
Training loss at step-30: 0.0412
Training loss at step-40: 0.0862
Training acc over Epoch-2: 0.9651
Test accuracy: 0.9625

Epoch-3
Training loss at step-0: 0.1940
Training loss at step-10: 0.0461
Training loss at step-20: 0.0648
Training loss at step-30: 0.1030
Training loss at step-40: 0.1690
Training acc over Epoch-3: 0.9737
Test accuracy: 0.9668

Epoch-4
Training loss at step-0: 0.0351
Training loss at step-10: 0.0365
Training l